In [1]:
import pandas as pd
from datetime import date

In [ ]:
# global recoding on age and pram on sex 
anon_data_global_recoding_pram = pd.read_excel("anon_data/anon_data_global_recoding_pram.xlsx")
# global recoding on age, global recoding on marital status 
anon_data_global2 = pd.read_excel("anon_data/anon_data_global2.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex 
anon_data_global2_pram = pd.read_excel("anon_data/anon_data_global2_pram.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex, local supression 
anon_data_global2_pram_supression = pd.read_excel("anon_data/anon_data_global2_pram_supression.xlsx")

results_data = pd.read_excel("data/public_data_resultsE.xlsx")
pub_data_register = pd.read_excel('data/public_data_registerE.xlsx')

c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


#### Testing if the anonymised datasets can be de-anonymised 

In [3]:
quasi_identifiers1 = ["age_group", "citizenship", "marital_status", "zip"]
quasi_identifiers2 = ["age_group", "citizenship", "marital_status", "zip", "sex"]

def k_anonymity_violations(data, quasi_identifiers, k_levels=[2, 3, 5]):
    # Group by quasi-identifiers to get the counts for each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Dictionary to store the number of rows violating each k-level
    violations = {}
    total_rows = len(data)

    for k in k_levels:
        # Count rows where the group size is less than k
        violating_rows = grouped[grouped['count'] < k]['count'].sum()
        # Store as a percentage for comparison with R's output
        violations[k] = violating_rows, (violating_rows / total_rows) * 100

    return violations

In [4]:
def identify_k_anonymity_violations(data, quasi_identifiers, k=2):
    # Group by quasi-identifiers to count each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Filter for combinations where the count is less than k (i.e., violate k-anonymity)
    violating_combinations = grouped[grouped['count'] < k]
    
    # Merge to find the original rows that match these violating combinations
    violations = data.merge(violating_combinations.drop(columns='count'), on=quasi_identifiers, how='inner')
    
    return violations

Global recoding on age, pram on sex

In [5]:
k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)
vioalting_rows_global1_pram = identify_k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status 

In [6]:
k_anonymity_violations(anon_data_global2, quasi_identifiers1)
vioalting_rows_global2 = identify_k_anonymity_violations(anon_data_global2, quasi_identifiers1)

Global recoding on age, global recoding on marital status, pram on sex

In [7]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)
vioalting_rows_global2_pram = identify_k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status, pram on sex, local supression 

In [13]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)


{2: (19, 9.5), 3: (37, 18.5), 5: (84, 42.0)}

#### Deanonymising the dataset 

##### Global recoding on age, global recoding on marital status, pram on sex

Prepare the public register voting data 
- convert dob to age 
- convert age to age groups 
- globally record marital status (like in anonymised data)

In [8]:
# convert dob to age 
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
pub_data_register["dob"] = pd.to_datetime(pub_data_register["dob"])
pub_data_register['age'] = pub_data_register['dob'].apply(lambda x: calculate_age(x))

# convert age to age groups 
age_bins = [18, 30, 38, 48, 58, 70, 101]  # Adjust as needed
age_labels = ["(18,30]", "(30,38]", "(38,48]", "(48,58]", "(58,70]", "(70,101]"] 
pub_data_register['age_group'] = pd.cut(pub_data_register['age'], bins=age_bins,labels= age_labels,right=True)

# define map 
status_map = {
    'Divorced': 'Not married',
    'Married/separated': 'Married',
    'Never married': 'Not married',
    'Widowed': 'Not married'
}
# Apply the map to the marital_status column
pub_data_register["marital_status"]= pub_data_register["marital_status"].map(status_map)
#pub_data_register["marital_status"]= pub_data_register["marital_status"].apply(lambda x: status_map(x))
pub_data_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age,age_group
0,"Turner, Destiny",Female,2001-04-01,2300,Latvia,Not married,2,23,"(18,30]"
1,"Smith, Ian",Male,1999-07-26,2200,Denmark,Not married,2,25,"(18,30]"
2,"Tucker, Kateland",Female,2001-06-19,2300,Denmark,Not married,1,23,"(18,30]"
3,"Robles Talavera, Troy",Male,2000-02-03,2100,Denmark,Not married,2,24,"(18,30]"
4,"Cordova, Lexis",Female,1999-08-25,2200,Denmark,Not married,2,25,"(18,30]"
...,...,...,...,...,...,...,...,...,...
1558,"Harris, J Shun",Male,1936-03-12,2300,Denmark,Not married,2,88,"(70,101]"
1559,"Todman, Alexis",Female,1968-03-01,2400,Denmark,Not married,0,56,"(48,58]"
1560,"el-Dar, Rushdi",Male,1951-11-07,2400,Denmark,Not married,0,73,"(70,101]"
1561,"el-Salehi, Shaahida",Female,1946-02-14,2200,Denmark,Married,2,78,"(70,101]"


Find the matching rows from the violating rows of the anonymised data i.e. the unqique rows in that dataset and print the quasi-identifiers + the name and the party voted for 

In [9]:
# important columns to select 
quasi_columns = quasi_identifiers1 + ["party", "name"]
# merge the violating rows and the public register dataset 
matched_df = vioalting_rows_global2_pram.merge(pub_data_register, on=quasi_identifiers1, how='inner')
# select only relevant columns 
matched_df[quasi_columns]

,age_group,citizenship,marital_status,zip,party,name
0,"(30,38]",Germany,Not married,2300,Green,"Liston, Kiet"
1,"(30,38]",Iraq,Married,2200,Green,"al-Morad, Aseela"
2,"(48,58]",Denmark,Married,2100,Green,"Yu, Chaelene"
3,"(48,58]",Denmark,Married,2100,Green,"Banerjee, Isaac"
4,"(48,58]",Denmark,Married,2100,Green,"Muppala, Emily"
5,"(48,58]",Denmark,Married,2100,Green,"Grabb, William"
6,"(48,58]",Denmark,Married,2100,Green,"el-Mahdavi, Shakeel"
7,"(48,58]",Denmark,Married,2100,Green,"Celestino, Zachary"
8,"(48,58]",Denmark,Married,2100,Green,"Bellow, Samuel"
9,"(48,58]",Denmark,Married,2100,Green,"el-Saah, Ruwaida"


In [10]:
pub_data_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age,age_group
0,"Turner, Destiny",Female,2001-04-01,2300,Latvia,Not married,2,23,"(18,30]"
1,"Smith, Ian",Male,1999-07-26,2200,Denmark,Not married,2,25,"(18,30]"
2,"Tucker, Kateland",Female,2001-06-19,2300,Denmark,Not married,1,23,"(18,30]"
3,"Robles Talavera, Troy",Male,2000-02-03,2100,Denmark,Not married,2,24,"(18,30]"
4,"Cordova, Lexis",Female,1999-08-25,2200,Denmark,Not married,2,25,"(18,30]"
...,...,...,...,...,...,...,...,...,...
1558,"Harris, J Shun",Male,1936-03-12,2300,Denmark,Not married,2,88,"(70,101]"
1559,"Todman, Alexis",Female,1968-03-01,2400,Denmark,Not married,0,56,"(48,58]"
1560,"el-Dar, Rushdi",Male,1951-11-07,2400,Denmark,Not married,0,73,"(70,101]"
1561,"el-Salehi, Shaahida",Female,1946-02-14,2200,Denmark,Married,2,78,"(70,101]"


In [11]:
quasi_identifiers1

['age_group', 'citizenship', 'marital_status', 'zip']